In [ ]:
import numpy as np

# Introduction to Qiskit

I assume you know what a qubit is and how to describe a system of qubits:
1. Two level quantum system
2. Used to store information
3. A system of $n$ qubits is described by a complex unit vector in a $2^n$ dimensional space (Hilbert space)

In this introduction we will cover the following topics until time runs out
1. Quantum gates and quantum circuits
1. Circuit execution on a local simulator
1. Circuit execution on IBM Quantum Experience
1. Technical considerations
1. Qiskit Runtime (brief)

## Quantum gates and quantum circuits

1. Quantum gates
2. Quantum circuits
3. Measurements
4. Parametrized circuits

### Quantum gates


The state of a qubit is described by a complex vector, 

$\begin{equation}|\psi\rangle = \begin{bmatrix}\alpha\ \\ \beta \end{bmatrix}\end{equation}$

such that $|\alpha|^2 + |\beta|^2 = 1$. Using Dirac's notation, we have $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$.

A quantum gate is a unitary transformation described by a complex matrix.<br>
For a quantum gate acting on a single qubit,

$\begin{equation}U = \begin{bmatrix}u_{00} & u_{01} \\ u_{10} & u_{11}\end{bmatrix}\end{equation}$ 

and $U^\dagger U = 1$.

When any $U$ can be expressed as a sequence of gates taken from a finite set of gates, we say that this set of quantum gates is universal. 

### Quantum circuits


A [quantum circuit](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit) is a sequence of quantum gates applied on one or more qubits. It is made of at least one __quantum register__ and can contain one or many __classical register(s)__.

 * A quantum register is a collection of qubits. We will usually use a single quantum register.

 * A classical register is a collection of bits that stores the result of a measurement. When measuring a qubit, you can specify in which bit you want to store the measurement outcome -- or rely on the default assignation!

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit import ClassicalRegister, QuantumRegister

In [ ]:
# A quantum circuit with 3 qubits (in a default quantum register), no classical bit
qc_3 = QuantumCircuit(3)
qc_3.draw('mpl', scale=0.7)

In [ ]:
# A quantum circuit with 2 qubits, 2 classical bits
qc_2_2 = QuantumCircuit(2, 2)
qc_2_2.draw('mpl', scale=0.7)

In [ ]:
# A quantum circuit with 2 registers of 2 qubits each,
# and one classical register with 2 classical bits
qr0 = QuantumRegister(2, 'qr0')
qr1 = QuantumRegister(2, 'qr1')
cr = ClassicalRegister(2, 'cr')

qc_22_2 = QuantumCircuit(qr0, qr1, cr)
qc_22_2.draw('mpl', scale=0.7)

Gates are applied from left to right.

All the gates defined by Qiskit have a corresponding method in the `QuantumCircuit` class.<br>
Let's look at a few examples.

In [ ]:
# The NOT gate
qc = QuantumCircuit(1)
qc.x(0)
qc.draw('mpl', scale=0.7)

In [ ]:
# The Hadamard gate
qr0 = QuantumRegister(2, "qr0")
qr1 = QuantumRegister(2, "qr1")
cr = ClassicalRegister(2, "cr")

qc = QuantumCircuit(qr0,qr1,cr)

qc.h(qr0)

qc.draw("mpl", scale=0.7)

In [ ]:
# The RX gate, which applies a rotation around the X-axis by a given angle
qc = QuantumCircuit(1)
qc.rx(np.pi/2, 0)  # rotation of PI/2
qc.draw('mpl', scale=0.7)

In [ ]:
# The CNOT gate
qc = QuantumCircuit(2)
qc.cx(0, 1)  # first arg is control qubit, second is target qubit
qc.draw('mpl', scale=0.7)

In [ ]:
# The SWAP gate
qc = QuantumCircuit(2)
qc.swap(0, 1)  # first arg is control qubit, second is target qubit
qc.draw('mpl', scale=0.7)

#### <span style="color:blue">Your turn!</span>

Use Qiskit to create the following circuit in the cell below.

<img src="images/quantum_circuit.png" alt="Quantum circuit" style="width:500px;"/>

In [ ]:
qc = QuantumCircuit(3)

qc.h(range(3))
qc.rx(np.pi/2,0)
qc.ry(np.pi/3,1)
qc.rz(np.pi/4,2)

qc.cx([0,1,2],[1,2,0])
qc.sdg(2)
qc.h([1,2])

qc.draw('mpl', scale=0.7)

### Measurements


A measurement is a non-unitary operation that reads the state of the qubits.

Born's rule tells us that for a qubit in state $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$, the probability that the measurement's outcome is 0 is $|\alpha|^2$, and we have $|\beta|^2$ for the probability that we measure 1.

As mentioned before, the measurement outcome is stored in a classical bit.

In [ ]:
# Measure all qubits and create a classical register with 1:1 mapping between
# qubits number and classical bit number
qc_3.measure_all()
qc_3.draw('mpl', scale=0.7)

In [ ]:
# Measure the qubits and store the outcome in the classical bits of an existing
# classical register
qc_2_2.measure(0, 0)
qc_2_2.measure(1, 1)
qc_2_2.draw('mpl', scale=0.7)

In [ ]:
# Measure the qubits of one quantum register and save the result in the classical bits
# of an existing classical register
qc_22_2.measure(qr1, cr)
qc_22_2.draw('mpl', scale=0.7)

### Parametrized circuits


1. `Parameter` and `ParameterVector`
2. Parameters binding

The vast majority of algorithms that run on today's quantum computers are _variational algorithms_.

A variational algorithm is a hybrid algorithm where:
 1. The quantum circuits have parametrized gates and are used to compute the expectation value of some observable.
 2. A classical computer updates the parameters of the quantum circuits in order to minimize a cost function. 

In [ ]:
from qiskit.circuit import Parameter

params = [Parameter('θ' + str(i)) for i in range(3)]
params

In [ ]:
qc_var = QuantumCircuit(3)

qc_var.h([0, 1, 2])
qc_var.rx(params[0], 0)
qc_var.ry(params[1], 1)
qc_var.rz(params[2], 2)
qc_var.cx(0, 1)
qc_var.cx(1, 2)
qc_var.cx(2, 0)
qc_var.h(1)
qc_var.sdg(2)
qc_var.h(2)

qc_var.measure_all()

qc_var.draw('mpl', scale=0.7)

In [ ]:
values = np.random.random(3)

params_dict = {k:v for (k, v) in zip(params, values)}
params_dict

In [ ]:
qc_binded = qc_var.bind_parameters(params_dict)
qc_binded.draw('mpl', scale=0.7)

## Circuit execution (on a local simulator)

1. First backend : Aer Simulator
2. First approach : `execute` function
3. Second approach : `Sampler` class

In [ ]:
# The quantum circuit that we will execute
qc_bell1 = QuantumCircuit(2)
qc_bell1.h(0)
qc_bell1.cx(0, 1)
qc_bell1.measure_all()

qc_bell1.draw('mpl', scale=0.7)

### First backend : Aer Simulator


[`Aer`](https://docs.quantum.ibm.com/api/qiskit/0.39/aer) provides quantum computing simulators with realistic noise models.<br>
When you use a simulator through the `qiskit_aer` module, the computation runs locally. 

In [ ]:
from qiskit_aer import AerSimulator

In [ ]:
simulator = AerSimulator()

### First approach : `execute` function


This method (and its 33 parameters!) is documented [here](https://docs.quantum.ibm.com/api/qiskit/execute).

It will be the default circuit execution approach during the winter school.

Two parameters are mandatory:
 1. the circuit(s)
 2. the backend (quantum computer or simulator)

 The `execute` method performs the computation asynchronously. It returns a `Job` instance.
 Retrieving the result from the job is blocking.

In [ ]:
from qiskit import execute

In [ ]:
job = execute(qc_bell1, simulator)
result = job.result()
counts = result.get_counts()
print(counts)

In [ ]:
from qiskit.visualization import plot_histogram

plot_histogram(counts, figsize=(3, 3))

### Second approach : `Sampler` class



The `Sampler` class calculates (quasi) probabilities of bitstrings from quantum circuits.

In [ ]:
from qiskit.primitives import Sampler, BackendSampler

In [ ]:
# Exact
sampler = Sampler()
job = sampler.run(circuits=[qc_bell1])
print(f'Sampling distribution: {job.result().quasi_dists[0]}')

In [ ]:
sampler = BackendSampler(simulator, options={'shots':2000})
job = sampler.run(circuits=[qc_bell1])
print(f'Sampling distribution: {job.result().quasi_dists[0]}')

In [ ]:
job.result().quasi_dists[0].binary_probabilities()

In [ ]:
qc_var.draw('mpl', scale=0.7)

In [ ]:
job = sampler.run([qc_var, qc_var], parameter_values=[[0,0,0], [1,1,1]])
result = job.result()
print(result.quasi_dists)

## Circuit execution on IBM Quantum Experience

1. Account and provider
1. Compute ressources
1. Job submission
1. Circuit bundling
1. More realistic simulation

<img src="images/iqx_interface.png" alt="IQX interface" style="width:500px;"/>

### Account and provider

1. Save and load account
2. Providers and quantum devices

<img src="images/providers.png" alt="Providers" style="width:500px;"/>

The method `IBMQ.save_account(<TOKEN>)` will store your access token under `$HOME/.qiskit/qiskitrc`.

In [ ]:
from qiskit_ibm_provider import IBMProvider

In [ ]:
# IBMProvider.save_account(token="...", overwrite=True)

In [ ]:
provider = IBMProvider()

In [ ]:
print(provider.instances())

In [ ]:
# The provider that you will use for the winter school
provider = IBMProvider(instance="pinq-quebec-hub/ecole-dhiver/qml-workshop")

### Compute ressources

In [ ]:
# List the available backends
provider.backends()

In [ ]:
ibm_simulator = provider.get_backend('ibmq_qasm_simulator')
quebec = provider.get_backend('ibm_quebec')

### Job submission

In [ ]:
job = execute(qc_bell1, ibm_simulator)
print(job.job_id())

In [ ]:
from qiskit.providers import JobStatus

if job.status() == JobStatus.DONE:
    # This call is blocking
    result = job.result()
    for key, value in result.to_dict().items():
        print(f"{key} : {value}")
    # print(result.to_dict())
else:
    print('Job not finished yet!')

When submitting a job to a real quantum computer, you enter a queue and the time it takes before you can retrieve the results varies a lot.

It's a good habit to save the job ID on disk. You can always use your provider's handle to query a specific job. 

In [ ]:
# Let's try on a real quantum computer!
# job_quebec = execute(qc_bell1, quebec)
# job_id = job_quebec.job_id()
# print(job_id)

In [ ]:
job_id = 'cppt0c9sp83g008y9e4g'
job_quebec = provider.retrieve_job(job_id)

In [ ]:
if job_quebec.status() == JobStatus.DONE:
    # This call is blocking
    result = job_quebec.result()
    for key, value in result.to_dict().items():
        print(f"{key} : {value}")
    # print(result.to_dict())
else:
    print('Job not finished yet!')

In [ ]:
# Job is done, let's look at the counts
counts = job_quebec.result().get_counts()
plot_histogram(counts, figsize=(3, 3))

### Circuits bundling

You can bundle multiple circuits in the same job, this way you don't have to enter the queue for every circuits!

Of course, there is a maximum number of circuits that can be packed in a single job and this is device dependent.

While we are at it, we can also look at the maximum number of shots per experiment supported.

In [ ]:
quebec_conf = quebec.configuration()

print(f'Maximum number of circuits per job on quebec: {quebec_conf.max_experiments}')
print(f'Maximum number of shots per experiment: {quebec_conf.max_shots}')

We will prepare the circuits corresponding to the 3 remaining Bell's pairs.

For this, we will use the `compose` method (more information [here](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#compose)).

In [ ]:
qc_bell2 = QuantumCircuit(2)
qc_bell2.x(0)
qc_bell2.barrier()
qc_bell2.compose(qc_bell1, inplace=True)

qc_bell2.draw('mpl', scale=0.7)

In [ ]:

qc_bell3 = QuantumCircuit(2)
qc_bell3.x(1)
qc_bell3.barrier()
qc_bell3.compose(qc_bell1, inplace=True)

qc_bell3.draw('mpl', scale=0.7)

In [ ]:
qc_bell4 = QuantumCircuit(2)
qc_bell4.x([0, 1])
qc_bell4.barrier()
qc_bell4.compose(qc_bell1, inplace=True)

qc_bell4.draw('mpl', scale=0.7)

In [ ]:
circuits = [qc_bell1, qc_bell2, qc_bell3, qc_bell4]
counts = execute(circuits, simulator).result().get_counts()

In [ ]:
print(counts)

### More realistic simulation

In [ ]:
quebec_simulator = AerSimulator.from_backend(quebec)

## Technical considerations

### Backend properties

* Qubit's calibration data
* Gate's calibration data

You can access a description of the `BackendProperties` class [here](https://docs.quantum.ibm.com/api/qiskit/qiskit.providers.models.BackendProperties).

In [ ]:
quebec_prop = quebec.properties()

#### Backend configuration

* Basis gates
* Coupling map
* Supported instructions
* Max experiments
* Max shots
* etc.

You can access a description of the `BackendConfiguration` class [here](https://docs.quantum.ibm.com/api/qiskit/qiskit.providers.models.BackendConfiguration).

In [ ]:
quebec_conf = quebec.configuration()

In [ ]:
# Any quantum computation executed by the quebec device must be expressed in terms
# of its basis set of gates.
quebec_conf.basis_gates

#### Circuit depth and width

The __circuit depth__ is a qualitative evaluation of "how long" the circuit is.<br>
Roughly, it counts the number of gates that cannot be applied in parallel.

It is "qualitative" since all gates do not take the same amount of time when they are executed on a quantum computer.

For example, let's look at the duration of a `NOT` gate on `ibm_quebec``.

In [ ]:
not_gate_duration = quebec_prop.gate_length('x', 0)

print(f'NOT gate duration: {(not_gate_duration * 1e6):.3f} µs')

But why do we care?

Because quantum computers are noisy!

Among the different types of noise are qubit's relaxation and dephasing.

Qubit's __relaxation__ refers to the physical relaxation of the qubit from the $|1\rangle$ (excited) state to the $|0\rangle$ (ground) state. Qubit relaxation is characterized by the relaxation time $T_1$.<br>
For a qubit prepared in state $|1\rangle$, the probability to measure the qubit in state $|1\rangle$ after a time $t$ is given by $\mathcal{P}(|1\rangle) = \exp{(-t/T_1)}$.

In [ ]:
t1_q0 = quebec_prop.t1(0)
print(f'Relaxation time for qubit 0: {(t1_q0 * 1e6):.1f} µs')
print(f'Time before the probability to measure the qubit in the 1 ' + 
      f'state falls below 90%: {(-1 * quebec_prop.t1(0) * np.log(0.9) * 1e6):.1f} µs')

Qubit's __dephasing__ refers to the loss of phase information in the qubit state.<br>
For a qubit prepared in state $|+\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$, the probability that the qubit's state will evolve toward a statistical mixture of the $|+\rangle$ and $|-\rangle$ states is characterized by $T_2$.  

In [ ]:
t2_q0 = quebec_prop.t2(0)
print(f'Dephasing time for qubit 0: {(t2_q0 * 1e6):.1f} µs')

Take home message: keep your circuits short!

The __width__ of a quantum circuit is also a concept that you might encounter, it simply refers to the number of qubits in your circuit.

### Connectivity and transpiling

[Transpilation](https://docs.quantum.ibm.com/api/qiskit/transpiler) is the process of rewriting a given input circuit to match the topology of a specific quantum device and its native gates. It can also optimize the circuit for execution on present day noisy quantum systems.

<img src="images/transpiling_core_steps.png" alt="Transpilation" width="500">

Let's look again at the circuit that we implemented for the exercice.

In [ ]:
qc = QuantumCircuit(3)

qc.h([0, 1, 2])
qc.rx(np.pi / 2, 0)
qc.ry(np.pi / 3, 1)
qc.rz(np.pi / 4, 2)
qc.cx(0, 1)
qc.cx(1, 2)
qc.cx(2, 0)
qc.h(1)
qc.sdg(2)
qc.h(2)

qc.measure_all()

qc.draw('mpl', scale=0.7)

In [ ]:
from qiskit import transpile

qc_transpiled = transpile(qc, coupling_map = quebec_conf.coupling_map)
# qc_transpiled = transpile(qc, backend=quebec)

qc_transpiled.draw('mpl', scale=0.7, idle_wires=False)

We see that given a given `coupling_map`, the qubit have been allocated to physical qubits in a different order at the start of the circuit. Also, the measurement place the results in specific classical bits. The introduction of `SWAP` is chaging the order of the qubits. Some one qubit gates have been by the native gates of the quebec device. The `SWAP` gate is necessary because qubit 0 is not connected to qubit 2 on the real device.

In [ ]:
from qiskit import transpile

qc_transpiled = transpile(qc, backend=quebec)

qc_transpiled.draw('mpl', scale=0.7, idle_wires=False)

We see that the gates `H`, `Rx`, `Ry` and `Sdg` have been replaced by the native gates of the quebec device.

Also, `CX` is not a native gate form `ibm_quebec`. This gate is actually decomposed using the two qubits interaction `ECR`.  

## Qiskit Runtime

Qiskit `Runtime` is a quantum computing service and programming model that allows users to optimize workloads and efficiently execute them on quantum systems at scale. For more info, you can read [this](https://quantum-computing.ibm.com/lab/docs/iql/runtime/). 

In particular, Qiskit Runtime defines its own `Sampler` implementation and provides an `Options` class to encapsulate all the possible options (see [here](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.options.Options.html)).

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(channel="ibm_quantum", token="...", set_as_default=True, overwrite=True)

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Estimator, Options
from qiskit.test.reference_circuits import ReferenceCircuits
from qiskit.circuit.library import RealAmplitudes
 
service = QiskitRuntimeService(channel="ibm_quantum")

options = Options(optimization_level=1)

with Session(service=service, backend="ibm_quebec") as session:
    # Submit a request to the Sampler primitive within the session.
    sampler = Sampler(session=session, options=options)
    job = sampler.run(circuits=qc_bell1)
    print(job.job_id())
    print(f"Sampler results: {job.result()}")
